In [1]:
!pip install -U ultralytics albumentations onnxslim onnxruntime-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 369.4/369.4 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.2/146.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.9 MB/s eta 0:00:00
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.19
    Uninstalling albucore-0.0.19:
      Successfully uninstalled albucore-0.0.19
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.20
    Uninstalling albumentations-

# Imports and configs

In [2]:
from ultralytics.data.augment import Albumentations
from ultralytics.utils import LOGGER, colorstr
from ultralytics.utils.metrics import Metric
from ultralytics import RTDETR, settings
from functools import partial
import albumentations as A
import numpy as np
import warnings
import random
import shutil
import torch
import yaml
import json
import os

warnings.filterwarnings("ignore")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
class CFG:
    dataset_path = "/kaggle/input/byu-locating-bacterial-flagellar-motors-2025/"
    train_image_path = os.path.join(dataset_path, "train")
    train_label_path = os.path.join(dataset_path, "train_labels.csv")

    debug = False

    seed = 42
    n_fold = 5
    current_fold = 0
    
    epochs = 1 if debug else 10
    image_size = 640
    es_patience = 7
    device = "0"

    yolo_dataset_path = f"/temp/dataset/fold_{current_fold}"
    yolo_yaml_path = f"/temp/dataset/fold_{current_fold}/dataset.yaml"
    yolo_model_name = "rtdetr-l.pt"

    use_wandb = False
    project = "byu-locating-bacterial-flagellar-motors"
    name = yolo_model_name.split(".")[0] + f"_fold_{current_fold}"

In [4]:
torch.manual_seed(CFG.seed)
np.random.seed(CFG.seed)
random.seed(CFG.seed)

In [5]:
settings.update({
    "runs_dir": "/temp/logs", 
    "tensorboard": False
})

if CFG.use_wandb:
    from kaggle_secrets import UserSecretsClient
    import wandb
    os.environ["WANDB_API_KEY"] = UserSecretsClient().get_secret("WANDB_API_KEY")
    settings.update({"wandb": True})

# Data preprocessing

In [6]:
if os.path.exists(CFG.yolo_dataset_path):
    shutil.rmtree(CFG.yolo_dataset_path)

shutil.copytree(f"/kaggle/input/byu-flagellar-motor-detection-1-preprocessing/dataset/fold_{CFG.current_fold}", CFG.yolo_dataset_path)
os.makedirs(CFG.project, exist_ok=True)

In [7]:
yaml_data = {
    "path": CFG.yolo_yaml_path,
    "train": os.path.join(CFG.yolo_dataset_path, "images", "train"),
    "val": os.path.join(CFG.yolo_dataset_path, "images", "val"),
    "names": {0: "motor"}
}

with open(CFG.yolo_yaml_path, "w") as f:
    yaml.dump(yaml_data, f)

# Training

In [8]:
def __init__(self, p=1.0):
    self.p = p
    self.transform = None
    prefix = colorstr("albumentations: ")

    try:
        spatial_transforms = {
            "Affine",
            "BBoxSafeRandomCrop",
            "CenterCrop",
            "CoarseDropout",
            "Crop",
            "CropAndPad",
            "CropNonEmptyMaskIfExists",
            "D4",
            "ElasticTransform",
            "Flip",
            "GridDistortion",
            "GridDropout",
            "HorizontalFlip",
            "Lambda",
            "LongestMaxSize",
            "MaskDropout",
            "MixUp",
            "Morphological",
            "NoOp",
            "OpticalDistortion",
            "PadIfNeeded",
            "Perspective",
            "PiecewiseAffine",
            "PixelDropout",
            "RandomCrop",
            "RandomCropFromBorders",
            "RandomGridShuffle",
            "RandomResizedCrop",
            "RandomRotate90",
            "RandomScale",
            "RandomSizedBBoxSafeCrop",
            "RandomSizedCrop",
            "Resize",
            "Rotate",
            "SafeRotate",
            "ShiftScaleRotate",
            "SmallestMaxSize",
            "Transpose",
            "VerticalFlip",
            "XYMasking",
        } 
        
        T = [
            A.HorizontalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.VerticalFlip(p=0.5)
        ]

        self.contains_spatial = any(transform.__class__.__name__ in spatial_transforms for transform in T)
        
        self.transform = (
            A.Compose(T, bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"], check_each_transform=True))
            if self.contains_spatial
            else A.Compose(T)
        )
        
        if hasattr(self.transform, "set_random_seed"):
            self.transform.set_random_seed(torch.initial_seed())
        
        LOGGER.info(prefix + ", ".join(f"{x}".replace("always_apply=False, ", "") for x in T if x.p))
        
    except Exception as e:
        LOGGER.info(f"{prefix}{e}")

Albumentations.__init__ = __init__

In [9]:
def fitness(self):
    return (5 * self.mp * self.mr) / (4 * self.mp + self.mr)

Metric.fitness = fitness

In [10]:
model = RTDETR(CFG.yolo_model_name)

results = model.train(
    data=CFG.yolo_yaml_path,
    epochs=CFG.epochs,
    batch=6,
    device=CFG.device,
    imgsz=CFG.image_size,
    optimizer='AdamW',
    lr0=1e-4,
    lrf=0.1,
    warmup_epochs=0,
    dropout=0.1,
    project=CFG.project,
    name=CFG.name,
    exist_ok=True,
    patience=CFG.es_patience,
    save=True,
    seed=CFG.seed,
    val=True,
    verbose=True
)

100%|██████████| 63.4M/63.4M [00:00<00:00, 99.4MB/s]


Ultralytics 8.3.151 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/temp/dataset/fold_0/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=rtdetr-l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=rtdetr-l_fold_0, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=7, perspective=0.0, plots=True

100%|██████████| 755k/755k [00:00<00:00, 17.9MB/s]


Overriding model.yaml nc=80 with nc=1
WARNING ⚠️ no model scale passed. Assuming scale='l'.

                   from  n    params  module                                       arguments                     
  0                  -1  1     25248  ultralytics.nn.modules.block.HGStem          [3, 32, 48]                   
  1                  -1  6    155072  ultralytics.nn.modules.block.HGBlock         [48, 48, 128, 3, 6]           
  2                  -1  1      1408  ultralytics.nn.modules.conv.DWConv           [128, 128, 3, 2, 1, False]    
  3                  -1  6    839296  ultralytics.nn.modules.block.HGBlock         [128, 96, 512, 3, 6]          
  4                  -1  1      5632  ultralytics.nn.modules.conv.DWConv           [512, 512, 3, 2, 1, False]    
  5                  -1  6   1695360  ultralytics.nn.modules.block.HGBlock         [512, 192, 1024, 5, 6, True, False]
  6                  -1  6   2055808  ultralytics.nn.modules.block.HGBlock         [1024, 192, 1024, 5, 

100%|██████████| 5.35M/5.35M [00:00<00:00, 75.1MB/s]


...

                   all       2065       1530       0.96      0.963      0.978      0.639

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


      10/10      5.98G     0.2877     0.3471    0.09735          3        640: 100%|██████████| 1378/1378 [13:00<00:00,  1.76it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 173/173 [00:46<00:00,  3.74it/s]


                   all       2065       1530      0.963      0.971      0.983       0.63

10 epochs completed in 2.302 hours.
Optimizer stripped from byu-locating-bacterial-flagellar-motors/rtdetr-l_fold_0/weights/last.pt, 66.1MB
Optimizer stripped from byu-locating-bacterial-flagellar-motors/rtdetr-l_fold_0/weights/best.pt, 66.1MB

Validating byu-locating-bacterial-flagellar-motors/rtdetr-l_fold_0/weights/best.pt...
Ultralytics 8.3.151 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
rt-detr-l summary: 302 layers, 31,985,795 parameters, 0 gradients, 103.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 173/173 [00:44<00:00,  3.86it/s]


                   all       2065       1530      0.963      0.971      0.983       0.63
Speed: 0.2ms preprocess, 18.5ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to byu-locating-bacterial-flagellar-motors/rtdetr-l_fold_0


In [11]:
results = model.val(verbose=False, save_json=True)

print(json.dumps({
    "ap": float(results.box.ap[0]),
    "ap50": float(results.box.ap50[0]),
    "f1": float(results.box.f1[0]),
    "map": float(results.box.map),
    "map50": float(results.box.map50),
    "map75": float(results.box.map75),
    "maps": float(results.box.maps[0]),
    "mp": float(results.box.mp),
    "mr": float(results.box.mr),
    "p": float(results.box.p[0]),
    "r": float(results.box.r[0]),
    "f2": (5 * float(results.box.mp) * float(results.box.mr)) / (4 * float(results.box.mp) + float(results.box.mr))
}, indent=4))

Ultralytics 8.3.151 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
rt-detr-l summary: 302 layers, 31,985,795 parameters, 0 gradients, 103.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2585.1±1423.6 MB/s, size: 395.2 KB)


val: Scanning /temp/dataset/fold_0/labels/val.cache... 2065 images, 535 backgrounds, 0 corrupt: 100%|██████████| 2065/2065 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 345/345 [00:52<00:00,  6.62it/s]


                   all       2065       1530      0.964      0.972      0.984      0.631
Speed: 0.2ms preprocess, 20.0ms inference, 0.0ms loss, 0.3ms postprocess per image
Saving byu-locating-bacterial-flagellar-motors/rtdetr-l_fold_0/predictions.json...
Results saved to byu-locating-bacterial-flagellar-motors/rtdetr-l_fold_0
{
    "ap": 0.6306843751942807,
    "ap50": 0.9837468011412751,
    "f1": 0.9678594870468475,
    "map": 0.6306843751942807,
    "map50": 0.9837468011412751,
    "map75": 0.7339922881644698,
    "maps": 0.6306843751942807,
    "mp": 0.9638569302805277,
    "mr": 0.9718954248366013,
    "p": 0.9638569302805277,
    "r": 0.9718954248366013,
    "f2": 0.9702770173164496
}


In [12]:
model.export(format='torchscript', imgsz=CFG.image_size, optimize=False, batch=8)

Ultralytics 8.3.151 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel CPUs. Learn more at https://docs.ultralytics.com/integrations/openvino/
rt-detr-l summary: 302 layers, 31,985,795 parameters, 0 gradients, 103.4 GFLOPs

PyTorch: starting from 'byu-locating-bacterial-flagellar-motors/rtdetr-l_fold_0/weights/best.pt' with input shape (8, 3, 640, 640) BCHW and output shape(s) (8, 300, 5) (63.1 MB)

TorchScript: starting export with torch 2.5.1+cu121...
TorchScript: export success ✅ 50.5s, saved as 'byu-locating-bacterial-flagellar-motors/rtdetr-l_fold_0/weights/best.torchscript' (123.0 MB)

Export complete (71.1s)
Results saved to /kaggle/working/byu-locating-bacterial-flagellar-motors/rtdetr-l_fold_0/weights
Predict:         yolo predict task=detect model=byu-locating-bacterial-flagellar-motors/rtdetr-l_fold_0/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=byu-locating-bacteri

'byu-locating-bacterial-flagellar-motors/rtdetr-l_fold_0/weights/best.torchscript'

# Cleanup

In [13]:
shutil.rmtree("wandb", ignore_errors=True)
try:
    os.remove("yolo11n.pt")
    os.remove(CFG.yolo_model_name)
except:
    pass